# Finding the Right Spot

Requirements:
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
- Account managers need to travel a lot.
- Everyone in the company is between 25 and 40, give them some place to go party.
- The office dog—"Pepe" needs a hairdresser every month. Ensure there's one not too far away.

In [1]:
#importing necessary libraries
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
import os
import requests
import json
from dotenv import load_dotenv
from pymongo import MongoClient
import geopandas as gpd
from functools import reduce
import operator
import geopandas as gpd

In [2]:
#functions
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

# Calculate Distance Between GPS Points

import math

def haversine(coord1, coord2):
    R = 6372800  # Earth radius in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))

In [3]:
#checking connection and getting the database 'companies'
conn = MongoClient("localhost:27017")
db = conn.get_database("ironhack")
collection = db.get_collection("companies")

In [4]:
c = collection

I filtered the database 'companies' to find those with offices in Dublin and which had raised over €1.0M. The dataframe below shows the results obtained.

### Why Dublin?
- **Low Taxes**
Thanks to the governmental effort to welcome new businesses and support creation of a more diverse job market, Dublin has the most attractive corporate tax rates in the world – just 12,5 %. R&D expenditure generates a 25% tax credit for offset against corporate taxes in addition to a tax deduction at 12.5% — this means that companies can claim a refund of €37.50 for every €100 worth of R&D expenditure.
- **Talented workforce**
Significantly cheaper than London, Dublin attracts young tech talent from all over the world. Dublin produces over 26 thousand graduates every year and is home to three internationally-ranked universities — Trinity College Dublin, University College Dublin and Dublin City University. That’s already a huge talent pool right there. Global Talent Competitiveness Index ranked Dublin among the top 10 cities for attracting and retaining talent.
- **Ireland Help**
Ireland’s Local Enterprise Office (LEO) spearheads business support for companies looking to relocate to Ireland through mentoring, training and grants ranging from €15,000 to help companies research market demand to €80,000 for expansion.

In [5]:
pd.DataFrame(c.find(
    {"$and": [
        {"offices.city": 'Dublin'},
        {"total_money_raised": {'$gt': '1.0'}}
    ]},
    {"name": 1, "rating": 1, "total_money_raised": 1, 'offices.latitude':1, 'offices.longitude':1}
))

,_id,name,total_money_raised,offices
0,52cdef7c4bab8bd6752986b7,eRepublik,€2.75M,"[{'latitude': 40.43094, 'longitude': -3.695289..."
1,52cdef7d4bab8bd675298963,Iceberg,€400k,"[{'latitude': 53.344104, 'longitude': -6.267494}]"
2,52cdef7d4bab8bd675298aec,Dial2Do,€1M,"[{'latitude': 53.34202, 'longitude': -6.250684}]"
3,52cdef7d4bab8bd675299e60,payByMobile,€750k,"[{'latitude': None, 'longitude': None}]"
4,52cdef7e4bab8bd67529b0e2,DediServe,€1M,"[{'latitude': None, 'longitude': None}, {'lati..."
5,52cdef7e4bab8bd67529b59e,Branded Payment Solutions,€100k,"[{'latitude': 53.344104, 'longitude': -6.26749..."
6,52cdef7e4bab8bd67529b913,Xiam,€1M,"[{'latitude': None, 'longitude': None}]"
7,52cdef7f4bab8bd67529c55b,Clavis Technology,€3.66M,"[{'latitude': None, 'longitude': None}, {'lati..."


I decided to focus on Clavis Technology location after having analyzed Dial2Do and Iceberg mainly because of the total money raised and given the 'products/services' offered by the company. By looking at the information available, I saw Clavis Tech as a successfull tech startup company - At this point, I thought about two options: 1) buy Clavis Tech offices for my new gaming company or 2) buy an office building close to that location.
Clavis Tech provides software products and services. The Company offers cloud-based electronic commerce intelligence and data quality performance management solutions. 

Now, let's see if the Clavis Tech office location matches my chosen requirements, which include:
- A starbucks coffee since it is very popular among employees
- Some place to party since workforce is aged between 25 and 40
- A good transportation network since they need to have easy access to the airport

In [6]:
dublin = {'type': 'Point', 'coordinates': [-6.25415, 53.32306]}
clavis_tech = {'type': 'Point', 'coordinates': [-6.258292, 53.346976]}

In [7]:
#map_dublin
map_dublin = folium.Map(location = [53.346976,-6.258292], zoom_start = 15)

In [8]:
#Adding Clavis Technologies (searched coordinates in google maps since the data was missing from companies' database)
#Location Clavis
clavis_lat =  53.346976
clavis_long = -6.258292

#creating the icon for Clavis
icono = Icon(color = "red",
             prefix = "fa",
             icon = "briefcase",
             icon_color = "black",
             tooltip = "Clavis Tech"
)
loc = [clavis_lat, clavis_long]
#saving the marker in one variable
marker_clavis = Marker(location = loc, icon = icono)
#let's add the marker to the original dublin map where we already had Dial2do location saved
marker_clavis.add_to(map_dublin)
map_dublin

As we can see above, the red suitcase marker represents Clavis Technology, the starting point that I have used to conduct my ideal office location based on the above mentioned requirements.

By using Foursquare API [https://foursquare.com/developers/apps], I was able to conduct different calls to find the above mentioned requirements. Additionally, I also studied other venues such as veterinarian, vegan restaurants etc, which can be found in the notebook called 'Analysing Dublin Office Location'. However, I decided not to include them since the results were not accurate.
I used Clavis Tech location coordinates as the starting point and analyzed the different venues from there.

## Starbucks

Let's take a look at the starbucks requirement. Further details can be found in 'Analysing Dublin Office Location' notebook. 

In [9]:
load_dotenv()

True

In [10]:
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

In [11]:
#url and Starbucks venue chain ID
enlace = 'https://api.foursquare.com/v2/venues/explore'
starbucks = "556f676fbd6a75a99038d8ec"

In [12]:
#params to be taken into account in calling FourSquare API
parametros = { "client_id" : tok1,
          "client_secret" : tok2,
          "v" : "20180323",
          "ll": f"{clavis_tech.get('coordinates')[1]},{clavis_tech.get('coordinates')[0]}",
          "query": "starbucks",
          "limit" : 10
}

In [13]:
resp = requests.get (url = enlace, params = parametros)
data = json.loads(resp.text)

In [14]:
decoding_data = data.get('response')
decoded = decoding_data.get('groups')[0]
starbucks = decoded.get('items')

In [15]:
mapa_nombre = ["venue","name"]
latitud = ["venue", "location","lat"]
longitud = ["venue","location","lng"]

In [16]:
new_list = []
for diccionario in starbucks:
    starbuckslist = {}
    starbuckslist["name"] = getFromDict(diccionario,mapa_nombre)
    starbuckslist["latitud"] = getFromDict(diccionario,latitud)
    starbuckslist["longitud"] = getFromDict(diccionario,longitud)
    new_list.append(starbuckslist)

In [17]:
dataframestarbucks = pd.DataFrame(new_list)
dataframestarbucks.head()

,name,latitud,longitud
0,Starbucks,53.347591,-6.259555
1,Starbucks,53.350007,-6.259033
2,Starbucks,53.349626,-6.261584
3,Starbucks,53.347545,-6.263520
4,Starbucks,53.346098,-6.259072


In [18]:
#Let's see which starbucks is closer to Clavis Tech
clavis_tech_coord = 53.346976,  -6.258292
starbucks = {
    'Starbucks_0': (53.347591,  -6.259555),
    'Starbucks_1': (53.350007,  -6.259033),
    'Starbucks_2': (53.349626, -6.261584),
    'Starbucks_3': (53.347545,  -6.263520),
    'Starbucks_4': (53.346098,  -6.259072)
}

for places, coord in starbucks.items():
    distance = haversine(clavis_tech_coord, coord)
    print(places, distance)
#We can see the closest Starbucks is c.100m away -> chosen starbucks

Starbucks_0 108.22086756471793
Starbucks_1 340.69816585905284
Starbucks_2 366.951514430256
Starbucks_3 352.8512306094474
Starbucks_4 110.5404164190214


In [19]:
#Adding closest Starbucks to dublin map
#Location Clavis
starbucks_lat =  53.347591
starbucks_long = -6.259555

#creating the icon for Starbucks
icono = Icon(color = "darkgreen",
             prefix = "fa",
             icon = "coffee",
             icon_color = "black",
             tooltip = "Starbucks"
)
loc = [starbucks_lat, starbucks_long]
#saving the marker in one variable
marker_starbucks = Marker(location = loc, icon = icono)
#let's add the marker to the map where we already had Clavis Tech location saved
marker_starbucks.add_to(map_dublin)
map_dublin

## Irish Pubs

Now, let's take a look at the 'party' requirement. Further details can be found in 'Analysing Dublin Office Location' notebook. 

In [20]:
enlace = 'https://api.foursquare.com/v2/venues/explore'
irish_pubs = "52e81612bcbc57f1066b7a06"
parametros = { "client_id" : tok1,
          "client_secret" : tok2,
          "v" : "20180323",
          "ll": f"{clavis_tech.get('coordinates')[1]},{clavis_tech.get('coordinates')[0]}",
          "query": "irish_pubs",
          "limit" : 10
}

In [21]:
resp = requests.get (url = enlace, params = parametros)
data = json.loads(resp.text)

In [22]:
decoding_data = data.get('response')
decoded = decoding_data.get('groups')[0]
irish_pubs = decoded.get('items')

In [23]:
mapa_nombre = ["venue","name"]
latitud = ["venue", "location","lat"]
longitud = ["venue","location","lng"]

In [24]:
new_list2 = []
for diccionario in irish_pubs:
    irishpubslist = {}
    irishpubslist["name"] = getFromDict(diccionario,mapa_nombre)
    irishpubslist["latitud"] = getFromDict(diccionario,latitud)
    irishpubslist["longitud"] = getFromDict(diccionario,longitud)
    new_list2.append(irishpubslist)
    
dataframeirish = pd.DataFrame(new_list2)
dataframeirish.head()

,name,latitud,longitud
0,Irish Whiskey Museum,53.344338,-6.259548
1,The Quays,53.345554,-6.263121
2,Irish Rock ‘N’ Roll Museum Experience,53.344976,-6.264444
3,Irish Film Institute (IFI),53.344667,-6.265295
4,Irish Design Shop,53.342667,-6.263078


In [25]:
#Let's now calculate distantes to irish pubs
clavis_tech_coord = 53.346976,  -6.258292
irish_pubs = {
    'Irish Whiskey Museum': (53.344338,  -6.259548),
    'The Quays': (53.345554,  -6.263121),
    'Irish Rock ‘N’ Roll Museum Experience': (53.344976, -6.264444),
    'Irish Film Institute (IFI)': (53.344667,  -6.265295),
    'Irish Design Shop': (53.342667,  -6.263078)
}

for places, coord in irish_pubs.items():
    distance = haversine(clavis_tech_coord, coord)
    print(places, distance)
#We can see there are many irish pubs nearby: Irish Whiskey Museum and the Quays less than 400m away

Irish Whiskey Museum 305.0374232836449
The Quays 357.5307752212451
Irish Rock ‘N’ Roll Museum Experience 465.13649321082136
Irish Film Institute (IFI) 531.2096372569788
Irish Design Shop 575.0654827285944


In [26]:
#Adding closest Irish pub to dublin map
#Location Irish Pub
pub_lat =  53.344338
pub_long = -6.259548

#creating the icon for Irish Pub
icono = Icon(color = "orange",
             prefix = "fa",
             icon = "beer",
             icon_color = "black",
             tooltip = "Irish Pub"
)
loc = [pub_lat, pub_long]
#saving the marker in one variable
marker_pub = Marker(location = loc, icon = icono)
#let's add the marker to the map where we already had Clavis Tech location saved
marker_pub.add_to(map_dublin)
map_dublin

## Veterinarian

In [27]:
#Looking for dog vet service
enlace = 'https://api.foursquare.com/v2/venues/explore'
vet = "4d954af4a243a5684765b473"
parametros = { "client_id" : tok1,
          "client_secret" : tok2,
          "v" : "20180323",
          "ll": f"{clavis_tech.get('coordinates')[1]},{clavis_tech.get('coordinates')[0]}",
          "query": "vet",
          "limit" : 10
}

In [29]:
resp = requests.get (url = enlace, params = parametros)
data = json.loads(resp.text)
decoding_data = data.get('response')
decoded = decoding_data.get('groups')[0]
vet = decoded.get('items')

In [30]:
new_list3 = []
for diccionario in vet:
    vetlist = {}
    vetlist["name"] = getFromDict(diccionario,mapa_nombre)
    vetlist["latitud"] = getFromDict(diccionario,latitud)
    vetlist["longitud"] = getFromDict(diccionario,longitud)
    new_list3.append(vetlist)
    
dataframevet = pd.DataFrame(new_list3)
dataframevet.head()

,name,latitud,longitud
0,The Porterhouse,53.345159,-6.267517
1,Duleek Vet Hospital,53.655193,-6.416615
2,Vet Farm Supplies,53.525459,-7.344558
3,Shannon Vet Surgery,53.423706,-7.983576


In [31]:
#Let's now calculate distantes to dog vet services
clavis_tech_coord = 53.346976,  -6.258292
vet_serv = {
    'The Porterhouse': (53.345159,  -6.267517),
    'Duleek Vet Hospital': (53.655193,  -6.416615),
    'Vet Farm Supplies': (53.525459, -7.344558),
    'Shannon Vet Surgery': (53.423706,  -7.983576)
}

for places, coord in vet_serv.items():
    distance = haversine(clavis_tech_coord, coord)
    print(places, distance)
#We can see there is a veterinarian where we could find a hairdresser for the dog (650m away)

The Porterhouse 645.0180090697779
Duleek Vet Hospital 35846.2829981018
Vet Farm Supplies 74662.12139205004
Shannon Vet Surgery 114768.11073931814


In [32]:
#Adding closest vet to dublin map
#Location Vet
vet_lat =  53.345159
vet_long = -6.267517

#creating the icon for Veterinarian
icono = Icon(color = "blue",
             prefix = "fa",
             icon = "paw",
             icon_color = "black",
             tooltip = "Veterinarian"
)
loc = [vet_lat, vet_long]
#saving the marker in one variable
marker_vet = Marker(location = loc, icon = icono)
#let's add the marker to the map where we the other locations and markers saved
marker_vet.add_to(map_dublin)
map_dublin

By taking a look at the map, I can see there is a good transportation network too, which will make the airport access easier and faster since the account managers need to travel a lot. In fact, Westmoreland tram station is only 80m away and there are other close tram stops nearby, such as the one in Tara St and in Trinity Luas.

## Conclusion

Clavis Tech office buildings seem to be the perfect spot for the new gaming company since it meets all the above mentioned requirements.The coordinates are: [-6.258292, 53.346976]. 